# Penentuan Negara Prioritas Ekspor-Impor Hasil Perikanan di Indonesia Dengan Menggunakan Algoritma A* dan Genetic Algorithm

Nama Anggota Kelompok :

1. Gerrard Sebastian - NIM 1203220018
2. Ali Rafli Putra Hakiki - NIM 1203220107
3. Okky Rangga Pratama - NIM 1203220011

# IMPORT LIBRARY

In [1]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
        'modsim.py')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from modsim import *

## BY ATRIBUT (A*)

In [3]:
import pandas as pd

def read_data(filename):
    country_data = {}
    try:
        df = pd.read_excel(filename)
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return country_data
    
    for index, row in df.iterrows():
        if pd.notna(row[0]):  
            country = str(row[0]).strip() 
            try:
                import_ton = float(row[1]) if pd.notna(row[1]) else 0
                export_ton = float(row[6]) if pd.notna(row[6]) else 0
                import_usd = float(row[13]) if pd.notna(row[13]) else 0
                export_usd = float(row[18]) if pd.notna(row[18]) else 0
                
            except ValueError:
                print(f"Invalid tonnage value in row: {index + 2}")
                continue
            
            country_data[country] = {
                'import_ton': import_ton,
                'export_ton': export_ton,
                'import_usd': import_usd,
                'export_usd': export_usd
            }
    return country_data

def top_countries_by_metric(country_data, metric, top_n=5):
    return sorted(country_data.items(), key=lambda x: x[1][metric], reverse=True)[:top_n]

def main():
    filename = 'dataFIXmerges.xlsx'
    country_data = read_data(filename)

    top_exporters_ton = top_countries_by_metric(country_data, 'export_ton')
    top_importers_ton = top_countries_by_metric(country_data, 'import_ton')
    top_exporters_usd = top_countries_by_metric(country_data, 'export_usd')
    top_importers_usd = top_countries_by_metric(country_data, 'import_usd')

    print("Top Exporters by Heavy (TON):")
    for country, data in top_exporters_ton:
        print(f"{country}: {data['export_ton']}")

    print("\nTop Importers by Heavy (TON):")
    for country, data in top_importers_ton:
        print(f"{country}: {data['import_ton']}")

    print("\nTop Exporters by Value (USD):")
    for country, data in top_exporters_usd:
        print(f"{country}: {data['export_usd']}")

    print("\nTop Importers by Value (USD):")
    for country, data in top_importers_usd:
        print(f"{country}: {data['import_usd']}")

if __name__ == "__main__":
    main()


Invalid tonnage value in row: 2
Invalid tonnage value in row: 3
Invalid tonnage value in row: 4
Invalid tonnage value in row: 5
Invalid tonnage value in row: 6
Invalid tonnage value in row: 7
Invalid tonnage value in row: 8
Invalid tonnage value in row: 9
Invalid tonnage value in row: 10
Invalid tonnage value in row: 11
Invalid tonnage value in row: 12
Invalid tonnage value in row: 13
Invalid tonnage value in row: 14
Invalid tonnage value in row: 15
Invalid tonnage value in row: 16
Invalid tonnage value in row: 17
Invalid tonnage value in row: 18
Invalid tonnage value in row: 19
Invalid tonnage value in row: 20
Invalid tonnage value in row: 21
Invalid tonnage value in row: 22
Invalid tonnage value in row: 23
Invalid tonnage value in row: 24
Invalid tonnage value in row: 25
Invalid tonnage value in row: 26
Invalid tonnage value in row: 27
Invalid tonnage value in row: 28
Invalid tonnage value in row: 29
Invalid tonnage value in row: 30
Invalid tonnage value in row: 31
Invalid tonnage va

C:\Users\gerra\AppData\Local\Temp\ipykernel_18076\4004144108.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notna(row[0]):
C:\Users\gerra\AppData\Local\Temp\ipykernel_18076\4004144108.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  country = str(row[0]).strip()
C:\Users\gerra\AppData\Local\Temp\ipykernel_18076\4004144108.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  import_ton = float(row[1]) if pd.notna(row[

## BY PRIORITY SCORE (A*)

In [4]:
import pandas as pd

def read_data(filename):
    country_data = {}
    try:
        df = pd.read_excel(filename)
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return country_data

    for index, row in df.iterrows():
        if pd.notna(row.iloc[0]):
            country = str(row.iloc[0]).strip()
            metrics = {}
            try:
                metrics['import_ton'] = float(row.iloc[1]) if pd.notna(row.iloc[1]) else 0
                metrics['export_ton'] = float(row.iloc[6]) if pd.notna(row.iloc[6]) else 0
                metrics['import_usd'] = float(row.iloc[13]) if pd.notna(row.iloc[13]) else 0
                metrics['export_usd'] = float(row.iloc[18]) if pd.notna(row.iloc[18]) else 0
                country_data[country] = metrics
            except ValueError as ve:
                print(f"Invalid value in row: {index + 2}, column causing error: {ve}")
                continue
    return country_data

def calculate_priority(country_data):
    # Define weights for each metric
    weights = {'import_ton': 1, 'export_ton': 2, 'import_usd': 3, 'export_usd': 4}
    for country, metrics in country_data.items():
        metrics['priority'] = (weights['import_ton'] * metrics['import_ton'] +
                               weights['export_ton'] * metrics['export_ton'] +
                               weights['import_usd'] * metrics['import_usd'] +
                               weights['export_usd'] * metrics['export_usd'])
    
    sorted_countries = sorted(country_data.items(), key=lambda x: x[1]['priority'], reverse=True)
    top_five = sorted_countries[:5]
    bottom_five = sorted_countries[-5:]
    return top_five, bottom_five

def main():
    filename = 'dataFIXmerges.xlsx'
    country_data = read_data(filename)
    top_five, bottom_five = calculate_priority(country_data)

    print("Top 5 Countries by Priority Score:")
    for country, data in top_five:
        print(f"{country}: Priority Score = {data['priority']}")

    print("\nBottom 5 Countries by Priority Score:")
    for country, data in bottom_five:
        print(f"{country}: Priority Score = {data['priority']}")

if __name__ == "__main__":
    main()


Invalid value in row: 2, column causing error: could not convert string to float: 'BENUA'
Invalid value in row: 3, column causing error: could not convert string to float: 'ASIA'
Invalid value in row: 4, column causing error: could not convert string to float: 'EROPA'
Invalid value in row: 5, column causing error: could not convert string to float: 'AFRIKA'
Invalid value in row: 6, column causing error: could not convert string to float: 'OSEANIA'
Invalid value in row: 7, column causing error: could not convert string to float: 'EROPA'
Invalid value in row: 8, column causing error: could not convert string to float: 'AFRIKA'
Invalid value in row: 9, column causing error: could not convert string to float: 'AMERIKA UTARA'
Invalid value in row: 10, column causing error: could not convert string to float: 'AMERIKA SELATAN'
Invalid value in row: 11, column causing error: could not convert string to float: 'ASIA'
Invalid value in row: 12, column causing error: could not convert string to fl

## BY FITNESS SCORE (GENETIC ALGORITHM)

In [5]:
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms

# Load the Excel file
file_path = 'dataFIXmerges.xlsx'
excel_data = pd.ExcelFile(file_path)

# Load data from Sheet1
sheet1_data = pd.read_excel(file_path, sheet_name='Sheet1')

# Clean and preprocess the data
sheet1_data = sheet1_data.drop(0)  # Remove the first row which seems to be a header for the columns
sheet1_data.columns = ['Country', 'Continent', 'Export_2018_Vol', 'Export_2019_Vol', 'Export_2020_Vol', 'Export_2021_Vol', 'Export_2022_Vol', 'Export_2023_Vol',
                       'Extra1', 'Extra2', 'Extra3', 'Extra4', 'Extra5', 'Extra6', 'Extra7', 'Extra8', 'Extra9', 'Extra10', 'IMPOR', 'Unnamed: 19', 'Unnamed: 20', 
                       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 
                       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34']

# Drop unnecessary columns with 'Extra' in their names and other unnamed columns
columns_to_drop = [col for col in sheet1_data if 'Extra' in col or 'Unnamed' in col]
sheet1_data = sheet1_data.drop(columns=columns_to_drop)

# Convert numerical columns to proper data types
for col in sheet1_data.columns[2:]:
    sheet1_data[col] = pd.to_numeric(sheet1_data[col], errors='coerce')

# Define the genetic algorithm components
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def create_individual():
    return creator.Individual(np.random.permutation(len(sheet1_data)).tolist())

def calculate_fitness(individual):
    total_export = sheet1_data.iloc[individual, 2:8].sum().sum()  # Export columns from 2018 to 2023
    total_import = sheet1_data.iloc[individual, -6:].sum().sum()  # Import columns from 2018 to 2023
    return total_export + total_import,

toolbox = base.Toolbox()
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", calculate_fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Genetic Algorithm parameters
population_size = 10
generations = 50
mutation_rate = 0.1

# Initialize the population
population = toolbox.population(n=population_size)

# Run the Genetic Algorithm
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=mutation_rate, ngen=generations, verbose=True)

# Get the best solution
best_individual = tools.selBest(population, k=1)[0]
best_fitness = calculate_fitness(best_individual)

# Get the data corresponding to the best individual
best_data = sheet1_data.iloc[best_individual].reset_index(drop=True)

# Add the total export and import as a new column
best_data['Total_Export_Import'] = best_data.iloc[:, 2:].sum(axis=1)

# Get top 5 countries by total export and import for each continent
top5_per_continent = best_data.groupby('Continent').apply(lambda x: x.nlargest(5, 'Total_Export_Import')).reset_index(drop=True)

# Save the top 5 per continent to a new Excel file
output_file_path = 'top5_exports_imports_by_continent.xlsx'
top5_per_continent.to_excel(output_file_path, index=False)

# Print the best solution and its fitness
print("Best Individual:", best_individual)
print("Best Fitness:", best_fitness[0])
print(f"Top 5 countries by export and import volume per continent saved to {output_file_path}")

# Display the cleaned data (optional)
print(sheet1_data)


gen	nevals
0  	10    
1  	4     
2  	5     
3  	6     
4  	10    
5  	6     
6  	8     
7  	8     
8  	9     
9  	4     
10 	4     
11 	8     
12 	8     
13 	4     
14 	6     
15 	8     
16 	6     
17 	5     
18 	8     
19 	7     
20 	6     
21 	8     
22 	4     
23 	6     
24 	7     
25 	6     
26 	6     
27 	6     
28 	6     
29 	8     
30 	10    
31 	8     
32 	8     
33 	6     
34 	8     
35 	6     
36 	8     
37 	4     
38 	10    
39 	6     
40 	10    
41 	6     
42 	7     
43 	10    
44 	10    
45 	4     
46 	8     
47 	6     
48 	9     
49 	9     
50 	6     
Best Individual: [104, 23, 174, 171, 102, 61, 49, 146, 67, 181, 89, 162, 15, 162, 210, 71, 73, 69, 134, 25, 163, 44, 178, 100, 95, 92, 9, 175, 115, 123, 125, 150, 106, 43, 10, 21, 36, 199, 5, 34, 72, 99, 159, 184, 59, 199, 164, 171, 22, 130, 62, 77, 9, 12, 17, 54, 31, 76, 25, 207, 76, 93, 63, 113, 161, 92, 182, 35, 117, 158, 172, 34, 185, 209, 11, 133, 171, 110, 191, 175, 125, 129, 178, 35, 169, 119, 82, 92, 184, 181, 130, 2

C:\Users\gerra\AppData\Local\Temp\ipykernel_18076\2315364634.py:70: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top5_per_continent = best_data.groupby('Continent').apply(lambda x: x.nlargest(5, 'Total_Export_Import')).reset_index(drop=True)
